In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import numpy as np
import pywt

import networkx as nx
import matplotlib.pyplot as plt
import folium
import geopandas as gpd

import itertools

# Get the current working directory + desired directory
parent_directory = os.getcwd()
main_directory = parent_directory + '/drive/MyDrive/2024 UW ENGINE Capstone'
curr_directory = main_directory + '/data/GMD and GIC'

In [ ]:
import os
import pandas as pd
import numpy as np
import pywt
import itertools
import networkx as nx
import matplotlib.pyplot as plt

eventDate = 20131002 # YYYYMMDD

# folder_path = 'C:\\Users\\19606\\Desktop\\GMDandGIC\\GICdata\\event_20131002\\GIC\\'

parent_directory = os.getcwd()
main_directory = parent_directory + '/drive/MyDrive/2024 UW ENGINE Capstone'
curr_directory = main_directory + '/data/GMD and GIC'
event_directory = curr_directory + '/event_' + str(eventDate) + '/GIC'

folder_path = event_directory

time_column = 'SampleDateTime'
gic_column = 'GICMeasured'

file_names = [f for f in os.listdir(event_directory) if f.startswith(str(eventDate)[0:4])]
print(file_names)

In [ ]:
loc_file = curr_directory + '/event_' + str(eventDate) + '/GIC/gic_monitors.csv'

latitude_file_path = loc_file

# latitude_file_path = 'C:\\Users\\19606\\Desktop\\GMDandGIC\\GICdata\\event_20131002\\gic_monitors.csv'
latitude_df = pd.read_csv(latitude_file_path)
latitude_dict = pd.Series(latitude_df[' Latitude'].values, index=latitude_df['Device ID']).to_dict()
latitude_dict

In [ ]:
def determine_max_level(data_length, wavelet='db4'):
    return pywt.swt_max_level(data_length)

def modwt(data, wavelet, level):
    return pywt.swt(data, wavelet, level=level, start_level=0)

def wavelet_cross_correlation(wt1, wt2):
    corrs = []
    for coeff1, coeff2 in zip(wt1, wt2):
        corr = np.corrcoef(coeff1[0], coeff2[0])[0, 1]
        corrs.append(corr)
    return corrs

def sliding_window_cross_correlation(wt1, wt2, window_size=30):

    assert len(wt1) == len(wt2)
    sliding_corrs = []

    for start in range(len(wt1) - window_size + 1):
        end = start + window_size
        window_corr = np.corrcoef(wt1[start:end], wt2[start:end])[0, 1]
        sliding_corrs.append(window_corr)

    return sliding_corrs

def make_length_even(data):
    return data if len(data) % 2 == 0 else np.append(data, data[-1])
edges = []

In [ ]:
modwt_results = {}

for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    df[time_column] = pd.to_datetime(df[time_column])
    df = df.drop_duplicates(subset=time_column).set_index(time_column)
    df = df.reindex(pd.date_range(start=df.index.min(), end=df.index.max(), freq='T'), method='nearest')
    resampled_df = df.resample('T').mean().interpolate(method='spline', order=3)
    gic_signal = make_length_even(resampled_df[gic_column].values)
    level = determine_max_level(len(gic_signal))
    modwt_result = modwt(gic_signal, 'db4', level)
    file_name = file_name.split('/')[-1]
    modwt_results[file_name] = modwt_result

In [ ]:
print("The wavelet transform coefficients of '2013E02_10056.csv' is:")
print(modwt_results['2013E02_10056.csv'])
print("The wavelet transform coefficients of '2013E02_10121.csv' is:")
print(modwt_results['2013E02_10121.csv'])

In [ ]:
desired_length = 1622

filtered_modwt_results = {file: modwt_vals for file, modwt_vals in modwt_results.items() if len(modwt_vals[0][0]) == desired_length}

filtered_modwt_results

In [ ]:
sliding_correlations = {}
for (file1, wt1), (file2, wt2) in itertools.combinations(filtered_modwt_results.items(), 2):
    corrs_over_time = sliding_window_cross_correlation(wt1[0][0], wt2[0][0])
    sliding_correlations[(file1, file2)] = corrs_over_time

In [ ]:
file_pair_to_visualize = (file_names[0], file_names[9])
correlations_over_time = sliding_correlations[file_pair_to_visualize]
time_axis = range(len(correlations_over_time))
print(time_axis)

In [ ]:
correlation_threshold = 0.8

node_degrees = {file_name: [0] * len(time_axis) for file_name in file_names}

for (file1, file2), correlations in sliding_correlations.items():
    for t, corr in enumerate(correlations):
        if abs(corr) > correlation_threshold:
            node_degrees[file1][t] += 1
            node_degrees[file2][t] += 1

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection
import numpy as np

cmap = plt.cm.hot

max_correlations = {}
for node in node_degrees:
    max_correlations[node] = np.full(len(time_axis), -np.inf)

for (node1, node2), correlations in sliding_correlations.items():
    for i, corr in enumerate(correlations):
        max_correlations[node1][i] = max(max_correlations[node1][i], corr)
        max_correlations[node2][i] = max(max_correlations[node2][i], corr)
max_correlations


In [ ]:
node_degrees = {file_name: [0] * len(time_axis) for file_name in file_names}

for (file1, file2), correlations in sliding_correlations.items():
    for t, corr in enumerate(correlations):
        if abs(corr) > correlation_threshold:
            node_degrees[file1][t] += 1
            node_degrees[file2][t] += 1
node_degrees_1 = {file_name.split('_')[-1].split('.')[0]: degrees
                        for file_name, degrees in node_degrees.items()}
node_degrees_1 = {int(key): value for key, value in node_degrees_1.items()}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

cmap = plt.cm.viridis
norm = plt.Normalize(vmin=0, vmax=1)

correlations_info = {node: {'values': [], 'exceeds_80_percentile': False} for node in node_degrees}

for (node1, node2), correlations in sliding_correlations.items():
    for i, corr in enumerate(correlations):
        if corr > 0.85:  # 只考虑超过0.85的相关性
            correlations_info[node1]['values'].append(corr)
            correlations_info[node2]['values'].append(corr)

# Determine which correlation values exceed the 80% threshold
for node, info in correlations_info.items():
    if info['values']:
        sorted_corrs = sorted(info['values'], reverse=True)

        percentile_index = int(np.ceil(0.8 * len(sorted_corrs))) - 1
        percentile_value = sorted_corrs[percentile_index]

        info['exceeds_80_percentile'] = sorted_corrs[0] > percentile_value

# draw the circle and set the color
fig, ax = plt.subplots(figsize=(14, 6))


for node, degrees_over_time in node_degrees.items():
    lat = latitude_dict[int(node.split('_')[-1].split('.')[0])]
    for time_idx, degree in enumerate(degrees_over_time):
        if correlations_info[node]['exceeds_80_percentile']:

            color = cmap(norm(correlations_info[node]['values'][0]))
        else:
            color = 'white'  # If the threshold of 80% is not exceeded, it is set to white
        circle = Circle((time_idx, lat), degree * 0.01, color=color, ec='black', lw=0.1)
        ax.add_patch(circle)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
fig.colorbar(sm, ax=ax)
ax.set_xlabel('Time Index')
ax.set_ylabel('Latitude')
ax.set_xlim(0, len(time_axis))
ax.set_ylim(min(latitude_dict.values()), max(latitude_dict.values()))

plt.show()


In [ ]:
correlations_info

In [ ]:
node_degrees = {file_name: [0] * len(time_axis) for file_name in file_names}

for (file1, file2), correlations in sliding_correlations.items():
    for t, corr in enumerate(correlations):
        if abs(corr) > correlation_threshold:
            node_degrees[file1][t] += 1
            node_degrees[file2][t] += 1
node_degrees = {file_name.split('_')[-1].split('.')[0]: degrees
                        for file_name, degrees in node_degrees.items()}
node_degrees = {int(key): value for key, value in node_degrees.items()}
max_correlations = {int(key.split('_')[-1].split('.')[0]): values
                             for key, values in max_correlations.items()}


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection
import numpy as np

# Setup the colormap
cmap = plt.cm.viridis
norm = plt.Normalize(vmin=np.nanmin([np.nanmin(v) for v in max_correlations.values() if np.isfinite(np.nanmin(v))]),
                     vmax=np.nanmax([np.nanmax(v) for v in max_correlations.values() if np.isfinite(np.nanmax(v))]))

fig, ax = plt.subplots(figsize=(45, 20))

# Create a circle for each node at each time point
for node, degrees_over_time in node_degrees.items():
    lat = latitude_dict[node]
    for time_idx, degree in enumerate(degrees_over_time):
        max_corr = max_correlations[node][time_idx]
        radius = degree * 0.01  # Adjust the factor for circle size as needed
        color = cmap(norm(max_corr)) if np.isfinite(max_corr) else 'white'
        # Add black edgecolor for better visibility
        circle = Circle((time_idx, lat), radius, color=color, ec='black', lw=0.1)  # lw is the linewidth of the edge
        ax.add_patch(circle)

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
fig.colorbar(sm, ax=ax)

# Set axis labels and limits
ax.set_xlabel('Time')
ax.set_ylabel('Latitude')
ax.set_xlim(0, len(time_axis))
ax.set_ylim(min(latitude_dict.values()), max(latitude_dict.values()))

plt.show()
